In [3]:
# %%
import os
import shutil
import json
from pathlib import Path

# === CONFIGURATION ===
# Manually define your paths
SOURCE_ROOT = Path("/home/debi/jaime/repos/MR-EyeTrack/results")
BIDS_ROOT = SOURCE_ROOT / "bids/derivatives/reconstructions"

# Gaze direction map
GAZE_MAP = {
    "0": "up",
    "1": "down",
    "2": "left",
    "3": "right"
}

# Undersampling map
UNDERSAMPLING_MAP = {
    "clean": "full",   # non-undersampled
    "clean_0.5": "50p",
    "clean_0.75": "75p",
    "clean_0.95": "95p"
}

# === MAIN SCRIPT ===
for sub_dir in sorted(SOURCE_ROOT.glob("Sub*")):
    subj_id = sub_dir.name.replace("Sub", "sub-")
    print(f"Processing {subj_id}...")

    # Create BIDS subject directory
    anat_dir = BIDS_ROOT / subj_id / "anat"
    anat_dir.mkdir(parents=True, exist_ok=True)

    # 1️⃣ Full reconstruction (no binning)
    full_recon = sub_dir / "T1_LIBRE_woBinning" / "output" / "th8" / "x_steva_nIter_20_delta_1.000.nii.gz"
    if full_recon.exists():
        bids_name = f"{subj_id}_rec-full_T1w.nii.gz"
        dest_path = anat_dir / bids_name
        shutil.copy(full_recon, dest_path)

        meta = {
            "ReconstructionDescription": "Full reconstruction using all readouts (no motion binning)",
            "FullReconstruction": True,
            "ReconstructionType": "static",
            "SkullStripped": False,
        }

        # -> .json, not .nii.json or .nii.gz.json
        json_path = dest_path.with_suffix("").with_suffix(".json")
        with open(json_path, "w") as f:
            json.dump(meta, f, indent=4)
    else:
        print(f"⚠️ Full reconstruction not found for {subj_id}")

    # 2️⃣ Motion-resolved reconstructions
    binning_root = sub_dir / "T1_LIBRE_Binning" / "output"

    for clean_folder, undersample_label in UNDERSAMPLING_MAP.items():
        folder = binning_root / clean_folder / "th8"
        if not folder.exists():
            continue

        for nii in folder.glob("x_steva_regionidx_*_nIter_20_delta_1.000.nii.gz"):
            region_idx = nii.name.split("regionidx_")[1].split("_")[0]
            gaze = GAZE_MAP.get(region_idx)
            if gaze is None:
                continue

            bids_name = f"{subj_id}_acq-{gaze}_rec-{undersample_label}_T1w.nii.gz"
            dest_path = anat_dir / bids_name
            shutil.copy(nii, dest_path)

            if dest_path.exists():  # only create JSON if copy succeeded
                meta = {
                    "ReconstructionDescription": (
                        f"Motion-resolved reconstruction for {gaze} gaze, "
                        f"{'non-undersampled' if undersample_label=='full' else undersample_label + ' undersampling'}"
                    ),
                    "GazeDirection": gaze,
                    "UndersamplingRate": (
                        None if undersample_label == "full" 
                        else float(undersample_label.replace('p','')) / 100.0
                    ),
                    "ReconstructionType": "motion-resolved",
                    "FullReconstruction": undersample_label == "full",
                    "SkullStripped": False,
                }

                # -> .json (not .nii.json)
                json_path = dest_path.with_suffix("").with_suffix(".json")
                with open(json_path, "w") as f:
                    json.dump(meta, f, indent=4)

print("\n✅ BIDS reconstruction dataset created under:", BIDS_ROOT)


Processing sub-001...
Processing sub-002...
Processing sub-003...

✅ BIDS reconstruction dataset created under: /home/debi/jaime/repos/MR-EyeTrack/results/bids/derivatives/reconstructions
